In [37]:
import os
path = "karanir.grammar"

grammar = ""
with open(path) as grammar_file:
    for line in grammar_file:
        grammar += line

grammar_ = """
start: instruction+

instruction: ("f"|"b"|"l"|"r") NUMBER
                | "c" COLOR [COLOR]
                | "fill" code_block
                | "repeat" NUMBER
code_block: "{" instruction+ "}"

COLOR: ("a".."z")+
NUMBER: ("0".."9")+ // one or more instructions

WHITESPACE: (" " | "\n")+
%ignore WHITESPACE
"""
print(grammar)


start: instruction+

instruction: ("f"|"b"|"l"|"r") NUMBER
                | "c" COLOR [COLOR]
                | "fill" code_block
                | "repeat" NUMBER
code_block: "{" instruction+ "}"

COLOR: ("a".."z")+
NUMBER: ("0".."9")+ // one or more instructions

WHITESPACE: (" " | "\n")+
%ignore WHITESPACE


In [64]:
from lark import Lark
parser = Lark(grammar)

text = """
c red yellow
c blue
(:predicate source)
"""

result = parser.parse(text)
print(result)

UnexpectedCharacters: No terminal matches '(' in the current parser context, at line 4 col 1

(:predicate source)
^
Expected one of: 
	* C
	* FILL
	* COLOR
	* B
	* F
	* REPEAT
	* R
	* L
